In [1]:
""" Purpose of this program is to load ticker/quote data into a local postgres database, eventually this may
be expanded to something like CosmosDB. Added bool columns for S&P, NASDAQ and DOW.
"""
import psycopg2
import pandas as pd
import yahoo_fin.stock_info as si
import time

In [60]:
conn = psycopg2.connect("dbname=StonksGoUp user=postgres host=localhost password=admin")
cur = conn.cursor()
data = si.get_data('msft')
SQL_stockdata = """ 
    CREATE TABLE stockdata (
        ticker varchar(5) NOT NULL,
        quotedate date NOT NULL,
        open numeric NOT NULL,
        high numeric NOT NULL,
        low numeric NOT NULL,
        close numeric NOT NULL,
        adjclose numeric NOT NULL,
        volume bigint,
        CONSTRAINT pk_stockdata PRIMARY KEY (ticker, quotedate)
);
"""
SQL_tickers = """
    CREATE TABLE tickers (
    ticker varchar(10) NOT NULL,
    isdow boolean NOT NULL DEFAULT false,
    isnasdaq boolean NOT NULL DEFAULT false,
    issp500 boolean NOT NULL DEFAULT false,
    createddate timestamp with time zone NOT NULL DEFAULT now(),
    CONSTRAINT pk_tickers PRIMARY KEY (ticker)
    );
"""
# cur.execute(SQL_stockdata, conn)
# cur.execute(SQL_tickers, conn)
# conn.commit()




In [51]:
"""INSERT data into the tickers reference table"""

df_ticker = pd.DataFrame()
nasdaq = si.tickers_nasdaq()
sp500 = si.tickers_sp500()
dow = si.tickers_dow()
other = si.tickers_other()

tickers = list(set([*nasdaq, *sp500, *dow, *other]))

df_ticker['ticker'] = tickers
df_ticker.replace("", float("NaN"), inplace=True)
df_ticker.dropna(subset = ["ticker"], inplace=True)

df_ticker['isdow'] = df_ticker['ticker'].isin(dow)
df_ticker['issp500'] = df_ticker['ticker'].isin(sp500)
df_ticker['isnasdaq'] = df_ticker['ticker'].isin(nasdaq)

insert = [list(row) for row in df_ticker.itertuples(index=False)]

SQL_Ticker_insert= """ INSERT INTO public.tickers(ticker,isdow, isnasdaq, issp500) VALUES (%s,%s,%s,%s) ON CONFLICT DO NOTHING"""
cur.executemany(SQL_Ticker_insert, insert)
conn.commit()

print(f'{cur.rowcount} rows inserted.')

In [32]:
"""INSERT quote data into stockdata table"""

SQL_stockdata_insert = """INSERT INTO public.stockdata (quotedate, open, high, low, close, adjclose, volume, ticker)
                            VALUES(%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT DO NOTHING"""
for ticker in df_ticker['ticker']:
    time.sleep(3)
    print(ticker)
    try: 
        data = si.get_data(ticker)
        insert_data = [list(row) for row in data.itertuples()]
        cur.executemany(SQL_stockdata_insert, insert_data)
    except: pass
    conn.commit()

In [59]:
cur.close()
conn.close()

In [65]:
tables = ('tickers', 'stocks')
SQL_check = f"""
                    SELECT tablename
                    FROM pg_catalog.pg_tables
                    WHERE schemaname != 'pg_catalog' 
                    AND schemaname != 'information_schema'
                    AND tablename IN {tables};
                    """

cur.execute(SQL_check, conn)
#tables_returned = cur.fetchall()
tables_returned = [x[0] for x in cur.fetchall()]
print(tables_returned)
#tables_returned - tables
list_diff = [i for i in tables + tables_returned if i not in tables or i not in tables_returned]
print(list_diff)

['tickers']


TypeError: can only concatenate tuple (not "list") to tuple